# <ins>Project Off-Target Fingerprinting</ins>

# Jupyter Notebook for **Results Processing**

This Jupyter notebook was used to filter the obtained docking scores and then transfer them into the matrix.

Beforehand, an Excel macro was executed which created a separate worksheet for each docked gridfile and separated the scores in the corresponding sheets (see VBA Scripts/SplitByGrid).

### Duplicate Filter

This cell filters the docking scores when multiple entries exist for the same molecule. Multiple entries occur when LigPrep generates several variants of the same molecule (e.g. different protonation states, tautomers, etc.)

In [ ]:
import os
from openpyxl import load_workbook
from collections import defaultdict

orig = 'input.xlsx'
sheet_names = [
    "1M2Z","1N83","1Q5K","1QPJ","1SQN","1Y2K","1Y57","2AM9","2AX9","2HI4","2NNJ","2PL0",
    "2Q8G","3E6I","3ERD","3HNG","3IBD","3L54","3NXU","3PBL","3V8S","3VO3","4AF3","4DJH",
    "4EBV","4EIY","4GQS","4IAR","4IB4","4LDE","4N6H","4OAR","4OD9","4U0I","4UHR","4WKQ",
    "5A5I","5E1S","5EW8","5H8N","5H8Q_allosteric_pocket","5IKR","5JFW","5K13","5OS5","5OSF",
    "5U09","5V54","5VEU","5XRA","5ZKC","5ZTY","6CM4","6DS0","6DZZ","6FO5","6IAR","6KUX",
    "6O4W","6PS2","6PT3","6WJC",
    "6X3X_GABA_pocket_alpha1_beta2","6X3X_benzo_pocket_alpha1_gamma2","6YG5","6YOJ","6YZ4","6ZFZ",
    "7AK0","7BU7","7BVQ","7CMV","7EJ8","7EU7","7F8Y","7JNT","7JVP","7JVR","7L28","7NG7","7NI4",
    "7S8O","7T8X","7TXT","7UL3","7WC6","7WC8","7XNN_elbow_pocket","7Y7Y","7Y7Z","7YM8","7YMJ",
    "8A27","8BLB","8DZR","8EA0","8EFO","8GUR","8JLZ","8PJK","8SSO",
    "8ST2_pocket_alpha_alpha","8ST2_pocket_alpha_beta","8WE8","8WEA","8XVK","8Y2F","8YN2","8YN3",
    "8ZP2","8ZYQ","9EO4","9GFE","9GYO","2V5Z","2Z5X","3RZE","3TBG","4LSJ",
    "5H8Q_glutamat_pocket","5H8Q_glycin_pocket","5I73_neu","6Y3C","7DTC","7F8W","7MBX",
    "7XNN_allosteric_pocket_chinidin","7XNN_pore_pocket_2","8FHS","8HCQ","8JBF",
    "8WGR_inward_open_allosteric_poc","8WGR_inward_open_central_pocket","8Y6H",
    "P37288","Q8TDR2"
]
base_name = 'input_without_duplicates'
ext = '.xlsx'

wb = load_workbook(orig, keep_vba=False)

total_deleted = 0

# remove duplicated
for sheet in sheet_names:
    ws = wb[sheet]
    print(f"Sheet {sheet}:")
    groups = defaultdict(list)
    for row in range(2, ws.max_row + 1):
        mol = ws[f'A{row}'].value
        cell = ws[f'F{row}'].value
        if mol is None or cell is None:
            continue
        if isinstance(cell, str):
            score_str = cell
            score_val = float(cell.replace(',', '.'))
        else:
            score_val = float(cell)
            score_str = str(cell).replace('.', ',')
        groups[mol].append((row, score_val, score_str))
    to_delete = []
    for mol, entries in groups.items():
        if len(entries) <= 1:
            continue
        best = min(entries, key=lambda e: e[1])
        removed = [e for e in entries if e[0] != best[0]]
        print(f"  {len(entries)}x {mol} | Kept: {best[2]} | Removed: " +
              "; ".join(e[2] for e in removed))
        to_delete += [e[0] for e in removed]
    print(f"Removed overall {len(to_delete)} duplicated \n")
    total_deleted += len(to_delete)
    for r in sorted(to_delete, reverse=True):
        ws.delete_rows(r, 1)

print(f"Removed over all sheets: {total_deleted} Duplicated\n")

# save as new file
new_name = base_name + ext
i = 1
while os.path.exists(new_name):
    new_name = f"{base_name}_{i}{ext}"
    i += 1

wb.save(new_name)
print(f"Saved as {new_name}")


### Results Transfer Tool

Transfers the docking scores into the matrix. Column selection is done either manually via user input or by loading a prepared CSV to avoid issues, as some PDB_IDs occur two or more times.

In case of already filled cells, the old value will be overwritten, when the new value is better (lower).

In [ ]:
import os
import re
import csv
import pandas as pd
from openpyxl import load_workbook

source_file = 'input_without_duplicates.xlsx'
target_file = 'matrix.xlsx'
base, ext   = 'matrix_filled', '.xlsx'
match_col   = 'B'
input_csv   = 'helper_csv_results_transfer_tool.csv' # note: csv without header

wb_tgt = load_workbook(target_file)
ws_tgt = wb_tgt.active

xls = pd.ExcelFile(source_file, engine='openpyxl')
sheet_names = [s for s in xls.sheet_names if s != 'All Entries']

mode = input("Select a csv file or manual input? Enter 'manual' or 'csv':\n> ").strip().lower()

zuordnung = []

if mode == 'csv':
    try:
        with open(input_csv, 'r', encoding='utf-8') as f:
            reader = csv.reader(f, delimiter=';')
            zuordnung = [row for row in reader if len(row) >= 2]
    except Exception as e:
        print(f"Error reading CSV: {e}")
        zuordnung = []
else:
    for sheet in sheet_names:
        col = input(f"To which column should the values from sheet {sheet} be transferred?")
        if col == '#':
            print("All remaining sheets are skipped.")
            break
        if col == '0':
            print(f"{sheet}: Skipped")
            continue
        if not re.fullmatch(r'[A-Z]+', col):
            print("Invalid input.")
            break
        zuordnung.append([sheet, col])

neueintragungen = []
ueberschreibungen = []
details_ueberschreibung = []

for sheet, col in zuordnung:
    if sheet not in sheet_names:
        print(f"Sheet '{sheet}' not found, will be kept.")
        continue

    df_src = pd.read_excel(source_file, sheet_name=sheet, engine='openpyxl', dtype=str)
    if 'molecule_name' not in df_src.columns or 'docking_score' not in df_src.columns:
        print(f"{sheet}: Removed columns, skip.\n")
        continue

    mapping = dict(zip(df_src['molecule_name'], df_src['docking_score']))
    mapping = {mol: val.replace('.', ',') for mol, val in mapping.items()}

    for row in range(2, ws_tgt.max_row + 1):
        mol = ws_tgt[f'{match_col}{row}'].value
        if mol not in mapping:
            continue

        new_val_str = mapping[mol]
        new_val = float(new_val_str.replace(',', '.'))
        cell = ws_tgt[f'{col}{row}']
        old_val_raw = cell.value

        if old_val_raw is None or str(old_val_raw).strip() == "":
            cell.value = new_val_str
            neueintragungen.append(cell.coordinate)
        else:
            try:
                old_val = float(str(old_val_raw).replace(',', '.'))
                if new_val < old_val:
                    cell.value = new_val_str
                    ueberschreibungen.append(cell.coordinate)
                    details_ueberschreibung.append(
                        f"{cell.coordinate} | Kept: {new_val_str} | Removed: {str(old_val_raw)}"
                    )
            except:
                continue

print("New entries in the following cells:")
print(", ".join(neueintragungen) if neueintragungen else "None")
print(f"Total number of new entries: {len(neueintragungen)}\n")

print("Overwrites in the following cells:")
print(", ".join(ueberschreibungen) if ueberschreibungen else "None")
print(f"Total number of overwrites: {len(ueberschreibungen)}")
if details_ueberschreibung:
    print("Details of overwritten values:")
    for line in details_ueberschreibung:
        print(line)

# save as new file
new_name = base + ext
i = 1
while os.path.exists(new_name):
    new_name = f"{base}_{i}{ext}"
    i += 1

wb_tgt.save(new_name)
print(f"\nSaved as {new_name}")
print(f"Path: {os.path.abspath(new_name)}")
